In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


def load_data(file_path):
    df = pd.read_csv(file_path)
    # Drop any rows with missing values
    df = df.dropna()
    # Convert the 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    # Sort the dataframe by date in ascending order
    df = df.sort_values('Date')
    return df


def preprocess_data(df):
    # Scale the 'Close' column to values between 0 and 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1, 1))
    return df


file_path = r'C:\Users\purva\Downloads\STOCK_INDEX.csv'
df = load_data(file_path)
df = preprocess_data(df)


In [14]:
from tensorflow.keras.models import load_model
import numpy as np


def make_predictions(model, test_data):
    last_day = test_data.iloc[-1]['Close']
    second_last_day = test_data.iloc[-2]['Close']
    predicted_prices = []

    # Predict the closing price for the last day
    last_day_pred = model.predict(np.array([[last_day]]))
    predicted_prices.append(last_day_pred[0][0])

    # Predict the closing price for the second last day
    second_last_day_pred = model.predict(np.array([[second_last_day]]))
    predicted_prices.append(second_last_day_pred[0][0])

    return predicted_prices


def evaluate(df, model):
    actual_close = np.loadtxt('sample_close.txt')
    pred_close = make_predictions(model, df)

    # Calculation of squared_error
    actual_close = np.array(actual_close)
    pred_close = np.array(pred_close)
    mean_square_error = np.mean(np.square(actual_close - pred_close))

    pred_prev = [df['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close

    actual_prev = [df['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr) - np.array(pred_prev)
    actual_dir = np.array(actual_curr) - np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir * actual_dir) > 0) * 100

    print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')


model = load_model(r'C:\Users\purva\Downloads\trained_model.h5')
evaluate(df, model)


1/1 [==============================] - 0s 12ms/step
Mean Square Error: 27199616.716133
Directional Accuracy: 100.0


In [15]:
def predict_func(df, model):
    return make_predictions(model, df)


next_day_pred = predict_func(df, model)[0]
next_next_day_pred = predict_func(df, model)[1]
print(f'Next Day Prediction: {next_day_pred}')
print(f'Next Next Day Prediction: {next_next_day_pred}')


1/1 [==============================] - 0s 14ms/step
Next Day Prediction: 2028.1351318359375
Next Next Day Prediction: 2051.3388671875
